In [1]:
# config
# config_name = 'config.yaml'
# config_name = 'LSTCrPPG.yaml'
config_name = 'PhysNet.yaml'
force_clear_train_cache = False
force_clear_test_cache = False

In [2]:
from singleton_pattern import load_model,load_data_generator,load_config
from singleton_pattern.load_dataset_reader import get_train_reader,get_test_reader
from singleton_pattern.load_data_generator import get_train_data_generator,get_test_data_generator
load_config.init_config(config_name)

from util.torch_info import print_info
print_info()
non_dnn_method_list = load_config.get_non_dnn_method_list()

config_hash = load_config.get_config_hash()
print(f'config_hash:{config_hash}')

config = load_config.get_config()
model = load_model.get_model()

print(f'Method and Preprocessing Name: {config["method"]}')
# print(f'Model:\n------------------\n{model}\n------------------')

CUDA:
---------------------------
PyTorch supports GPU and currently has 1 CUDA devices available.
The current default CUDA device is: NVIDIA GeForce RTX 3090
---------------------------
config_hash:b9beae6638c3f109c18742aedb2d1be048f5ff9adb50d2948c9f42f308696abe
Method and Preprocessing Name: PhysNet


In [3]:
import datetime
dataloader = None
def load_train_loader():
    global dataloader
    if dataloader != None:
        return dataloader
    train_reader = get_train_reader()
    print(f'train_reader: {train_reader.loader_name}')
    data = train_reader.load_data()
    tensor_data_generator = get_train_data_generator()
    dataloader = tensor_data_generator.get_tensor_dataloader(data,force_clear_cache=force_clear_train_cache)
    return dataloader
def train():
    if config['method'] in non_dnn_method_list: 
        print('non train')
        return
    dataloader = load_train_loader()
    start_time = datetime.datetime.now()
    model.train_model(dataloader)
    runtime = datetime.datetime.now() - start_time
    hours, remainder = divmod(runtime.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"train total time:  {int(hours)}h:{int(minutes)}m:{int(seconds)}s")
# run train
train()

train_reader: UBFC_Phys
Root Path:D:\BaiduNetdiskDownload\UBFC-PHYs_1-12
Start Reading UBFC_Phys Dataset Directory...


Progress:   0%|          | 0/12 [00:00<?, ?it/s]

cache\b9beae6638c3f109c18742aedb2d1be048f5ff9adb50d2948c9f42f308696abe\train
Start Generator Data...


Progress:   0%|          | 0/12 [00:00<?, ?it/s]

Processing videos 1 :   0%|          | 0/6326 [00:00<?, ?it/s]

In [ ]:
from method.LSTCrPPG import LSTCrPPG
from method.PhysNet import PhysNet
import torch
input_data = torch.randn(3, 3, 160, 128, 128)
model = PhysNet()

model.eval()
model(input_data)


In [ ]:
test_reader = get_test_reader()
tensor_data_generator = get_test_data_generator()
print(f'Test Loader: {test_reader.loader_name}')
# test
model.eval()

data = test_reader.load_data()
test_dataloader = tensor_data_generator.get_tensor_dataloader(data,force_clear_cache=force_clear_test_cache)

# X,y = model.preprocessing(test_df)
# pred_y = model.forward(X)

# 评价指标
